# Data copying vs. Data from pointers

If you have never programmed in C++ before (or even if you have), you may not be familiar with pointers. While it's beyond the scope of this tutorial to cover what pointers are and how they are used, this tutorial will briefly cover some of the issues you may have in Freud which involve pointers. If you are interested in pointers, or if you plan on developing freud, the [C++ tutorial on pointers](http://www.cplusplus.com/doc/tutorial/pointers/) is a good place to start.

## Python level example (not really pointers, but it gets the point across)

Run the following code and take a guess what the value of `b[0]` will be:
* 1
* 2

In [9]:
a = [1]
b = a
a[0] = 2
print(b[0])

2


## Variables by reference

In python, this is an example of assignment by reference. `b` is not created as its own list; rather, b is created as a reference to `a`, so by changing `a`, we also change `b`. In C++ you can do the same thing (create/pass variables by reference) but you can also do it by pointer, which is the memory address of the value. While there is a difference, the end result is the same: by changing the value of `a` you change the value of `b`. There are many pros to donig things this way, including lower memory usage and faster performance, but it's easy to inadvertantly overwrite data.

# Creating Numpy  arrays by pointer

In Freud we decided to create our numpy arrays by passing a pointer from C++. This was the fastest, most efficient way to do this, but can result in data being changed or overwritten, so take care when performing calculations, and create copies where necessary:

In [11]:
%matplotlib inline
import numpy as np
from cycler import cycler
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import LinearLocator, FormatStrFormatter, MultipleLocator, ScalarFormatter
from matplotlib.colors import colorConverter
import glotzformats
# these are settings I enjoy, feel free to use your own
myHexColors = ["#30A2DA", "#FC4F30", "#E5AE38", "#6D904F", "#9757DB",
               "#188487", "#FF7F00", "#9A2C66", "#626DDA", "#8B8B8B"]
myColorCycle = [colorConverter.to_rgb(i) for i in myHexColors]
my_marker_cycle = ["o", "^", "s", "p", "D", "v", "8", "h", "+", "."]
plt.rc('axes', prop_cycle=(cycler("color", myColorCycle) + 
                           cycler("marker", my_marker_cycle)))
from freud import parallel, box, density
parallel.setNumThreads(4)
gsd_reader = glotzformats.reader.GSDHOOMDFileReader()
rdf = density.RDF(rmax=10.0, dr=0.1)
data_path = "/nfs/glotzer/projects/EntropicBonding/2016/10/04102016/DataP0.55000/Run0/EntropicBonding.gsd"
# reset the rdf; required if not using compute
with open(data_path, "rb") as gsd_file:
    traj = gsd_reader.read(gsd_file)
    # for all frames except the first (your syntax will vary based on your reader)
    for frame in traj[1:]:
        # create the box
        gbox = frame.box
        fbox = box.Box(Lx=gbox.Lx, Ly=gbox.Ly, is2D=True)
        # get the positions
        pos = frame.positions
        # compute
        rdf.accumulate(fbox, pos, pos)

# get the center of the histogram bins
r_avg = rdf.getR()
# get the value of the histogram bins
y_avg = rdf.getRDF()
with open(data_path, "rb") as gsd_file:
    traj = gsd_reader.read(gsd_file)
    # for all frames except the first (your syntax will vary based on your reader)
    frame = traj[-1]
    # create the box
    gbox = frame.box
    fbox = box.Box(Lx=gbox.Lx, Ly=gbox.Ly, is2D=True)
    # get the positions
    pos = frame.positions
    # compute
    rdf.compute(fbox, pos, pos)
# get the center of the histogram bins
r = rdf.getR()
# get the value of the histogram bins
y = rdf.getRDF()
# create matplotlib plot
fig = plt.figure(figsize=(6,6))
# create axes
ax = fig.add_subplot(111)
ax.plot(r, y, marker=",", label=r"$g\left(r\right)$")
ax.plot(r_avg, y_avg, marker=",", label=r"$g\left(r\right)_{avg}$")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc=0, frameon=True, fontsize=16,
          ncol=1, fancybox=True, framealpha=0.5)
ax.set_xlabel(r"$r$")
ax.set_ylabel(r"$g \left( r \right)$")
ax.xaxis.label.set_fontsize(20)
ax.yaxis.label.set_fontsize(20)
ax.xaxis.set_tick_params(labelsize=16)
ax.yaxis.set_tick_params(labelsize=16)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/nfs/glotzer/projects/EntropicBonding/2016/10/04102016/DataP0.55000/Run0/EntropicBonding.gsd'